<a href="https://colab.research.google.com/github/RahulSundar/CS6910-DeepLearningFundamentals/blob/main/Assignment2/Assignment2_ObjectDetection_unbalanced_sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

from tensorflow.python.client import device_lib
device_lib.list_local_devices()



Found GPU at: /device:GPU:0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1546758841907655021, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15505193728
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8846710379610015337
 physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"]

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
1.5882476120000035
GPU (s):
0.046879610000004845
GPU speedup over CPU: 33x


In [3]:
!cat /proc/meminfo

MemTotal:       26751700 kB
MemFree:        18985140 kB
MemAvailable:   24581916 kB
Buffers:          103256 kB
Cached:          4692804 kB
SwapCached:            0 kB
Active:          2001836 kB
Inactive:        5256336 kB
Active(anon):    1400368 kB
Inactive(anon):    12716 kB
Active(file):     601468 kB
Inactive(file):  5243620 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              2840 kB
Writeback:             0 kB
AnonPages:       2462264 kB
Mapped:          1349940 kB
Shmem:             13324 kB
Slab:             218096 kB
SReclaimable:     148936 kB
SUnreclaim:        69160 kB
KernelStack:        5184 kB
PageTables:        14744 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    13375848 kB
Committed_AS:    6196736 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             2096 kB
AnonHugePages:   

In [4]:
!pip install wandb

!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip nature_12K.zip
!rm nature_12K.zip

!mv ./inaturalist_12K/val ./inaturalist_12K/test

#Commented out IPython magic to ensure Python compatibility.
%mkdir ./inaturalist_12K/eval

Streaming output truncated to the last 5000 lines.
  inflating: inaturalist_12K/train/Arachnida/1ac25349af662e85f577a31f64beca41.jpg  
  inflating: inaturalist_12K/train/Arachnida/117ef7f1812725f359500448a258e713.jpg  
  inflating: inaturalist_12K/train/Arachnida/6409254954c3b46d88d64f0bf4c632fa.jpg  
  inflating: inaturalist_12K/train/Arachnida/aaed865750bd52944658a7dd23366948.jpg  
  inflating: inaturalist_12K/train/Arachnida/460cc09ef510d400a4d47a836ddd45e6.jpg  
  inflating: inaturalist_12K/train/Arachnida/55c48c3afef0b745e53951284cdb2db4.jpg  
  inflating: inaturalist_12K/train/Arachnida/bff3daa961ee96a37ebe12dc0bcce504.jpg  
  inflating: inaturalist_12K/train/Arachnida/9082d4f2107bccf1f14d71c8944be607.jpg  
  inflating: inaturalist_12K/train/Arachnida/93d0130423d42f7c6228150eb95a2a93.jpg  
  inflating: inaturalist_12K/train/Arachnida/051aba6d7bc324bad76bd438ed3cc276.jpg  
  inflating: inaturalist_12K/train/Arachnida/67595af09c015c7753d5d4935677590f.jpg  
  inflating: inaturalist_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, sys
sys.path.append(
"/content/drive/MyDrive/CS6910/Assignment2"
)


In [51]:
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#import keras

# keras pre-trained models
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2 as IRV2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception


from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Conv2D, BatchNormalization, MaxPooling2D, Activation , GlobalAveragePooling2D
from tensorflow.keras.models import Sequential,  Model

import wandb


class ObjectDetection():

    def __init__(self, IMG_SIZE, modelConfigDict, using_pretrained_model = False, base_model = "IRV2" ):
        
        self.num_hidden_cnn_layers= modelConfigDict["num_hidden_cnn_layers"]
        self.activation = modelConfigDict["activation"]
        self.batch_normalization = modelConfigDict["batch_normalization"]
        self.filter_distribution = modelConfigDict["filter_distribution"]
        self.filter_size = modelConfigDict["filter_size"]
        self.number_of_filters_base  = modelConfigDict["number_of_filters_base"]
        self.dropout_fraction = modelConfigDict["dropout_fraction"]
        self.pool_size = modelConfigDict["pool_size"]
        self.padding = modelConfigDict["padding"]
        self.dense_neurons = modelConfigDict["dense_neurons"]
        self.num_classes = modelConfigDict["num_classes"]
        self.optimizer = modelConfigDict["optimizer"]
        self.global_average_pooling = modelConfigDict["global_average_pooling"]
        self.batch_normalisation_location = modelConfigDict["batch_normalisation_location"]
        BASE_MODELS = {
                          "IRV2" : IRV2,
                          "IV3" : InceptionV3,
                          "RN50" : ResNet50,
                          "XCPTN" : Xception
                      }      
        
        if using_pretrained_model == True:
            self.base_model = base_model
            if self.base_model == "RN50":
                self.IMG_HEIGHT = 224
                self.IMG_WIDTH = 224
            else:
                self.IMG_HEIGHT = IMG_SIZE[0]
                self.IMG_WIDTH = IMG_SIZE[1]        

        self.IMG_HEIGHT = IMG_SIZE[0]
        self.IMG_WIDTH = IMG_SIZE[1]        
         
        self.input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)


    def build_cnnmodel_conv(self):
        with tf.device('/device:GPU:0'):
            tf.keras.backend.clear_session()
            model = Sequential()
            
            #First CNN layer connecting to input layer
            model.add(Conv2D(self.number_of_filters_base, self.filter_size, padding = self.padding,kernel_initializer = "he_uniform", input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)))
            if self.batch_normalisation_location == "Before" and self.batch_normalization: model.add(BatchNormalization())
            model.add(Activation(self.activation))
            
            #batch_normalisation
            if self.batch_normalisation_location == "After" and self.batch_normalization: model.add(BatchNormalization())
            #max pooling
            model.add(MaxPooling2D(pool_size=self.pool_size))  
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            for i in range(self.num_hidden_cnn_layers-1):
                #i+2th Convolutional Layer
            
                ## Standard filter distribution - same number of filters in all Convolutional layers
                if self.filter_distribution == "standard":
                    model.add(Conv2D(self.number_of_filters_base, self.filter_size,kernel_initializer = "he_uniform",padding = self.padding))
            
                ## Double filter distribution - double number of filters in each Convolutional layers
                elif self.filter_distribution == "double":
                    model.add(Conv2D(2**(i+1)*self.number_of_filters_base, self.filter_size,kernel_initializer = "he_uniform", padding = self.padding))
            
                ## Halve the filter size in each successive convolutional layers
                elif self.filter_distribution == "half":
                    model.add(Conv2D(int(self.number_of_filters_base/2**(i+1)), self.filter_size,kernel_initializer = "he_uniform", padding = self.padding))
            
                if self.batch_normalisation_location == "Before" and self.batch_normalization: model.add(BatchNormalization())
                model.add(Activation(self.activation))
            
                if self.batch_normalisation_location == "After" and self.batch_normalization: model.add(BatchNormalization())
            
                model.add(MaxPooling2D(pool_size=self.pool_size))
                if self.dropout_fraction != None:
                    model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            
            #Final densely connected layers
            if self.global_average_pooling == True:
                model.add(GlobalAveragePooling2D())
            else:
                model.add(Flatten())

            model.add(Dense(self.dense_neurons, activation = 'sigmoid'))
            model.add(Dense(self.num_classes, activation = 'softmax'))

            return model      
        
    def build_cnnmodel_all(self):
        with tf.device('/device:GPU:0'):
            
            tf.keras.backend.clear_session()
            model = Sequential()
            
            #First CNN layer connecting to input layer
            model.add(Conv2D(self.number_of_filters_base, self.filter_size, padding = self.padding,kernel_initializer = "he_uniform", input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)))
            model.add(Activation(self.activation))
            
            #batch_normalisation
            if self.batch_normalisation_location == "After" and self.batch_normalization: model.add(BatchNormalization())
            #max pooling
            model.add(MaxPooling2D(pool_size=self.pool_size))  
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            for i in range(self.num_hidden_cnn_layers-1):
                #i+2th Convolutional Layer
            
                ## Standard filter distribution - same number of filters in all Convolutional layers
                if self.filter_distribution == "standard":
                    model.add(Conv2D(self.number_of_filters_base, self.filter_size,kernel_initializer = "he_uniform", padding = self.padding))
            
                ## Double filter distribution - double number of filters in each Convolutional layers
                elif self.filter_distribution == "double":
                    model.add(Conv2D(2**(i+1)*self.number_of_filters_base, self.filter_size,kernel_initializer = "he_uniform",padding = self.padding))
            
                ## Halve the filter size in each successive convolutional layers
                elif self.filter_distribution == "half":
                    model.add(Conv2D(int(self.number_of_filters_base/2**(i+1)), self.filter_size,kernel_initializer = "he_uniform", padding = self.padding))
            
                model.add(Activation(self.activation))
            
                if self.batch_normalisation_location == "After" and self.batch_normalization: model.add(BatchNormalization())
            
                model.add(MaxPooling2D(pool_size=self.pool_size))
                if self.dropout_fraction != None:
                    model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            
            #Final densely connected layers
            if self.global_average_pooling == True:
                model.add(GlobalAveragePooling2D())
            else:
                model.add(Flatten())

            model.add(Dense(self.dense_neurons, activation = 'sigmoid'))
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            model.add(Dense(self.num_classes, activation = 'softmax'))
            
            return model      
      
    def build_cnnmodel_dense(self):
        with tf.device('/device:GPU:0'):
            tf.keras.backend.clear_session()
            model = Sequential()
            
            #First CNN layer connecting to input layer
            model.add(Conv2D(self.number_of_filters_base, self.filter_size ,kernel_initializer = "he_uniform",padding = self.padding,input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)))
            if self.batch_normalisation_location == "Before" and self.batch_normalization: model.add(BatchNormalization())
            model.add(Activation(self.activation))
            
            #batch_normalisation
            if self.batch_normalisation_location == "After" and self.batch_normalization: model.add(BatchNormalization())
            #max pooling
            model.add(MaxPooling2D(pool_size=self.pool_size))  
            for i in range(self.num_hidden_cnn_layers-1):
                #i+2th Convolutional Layer
            
                ## Standard filter distribution - same number of filters in all Convolutional layers
                if self.filter_distribution == "standard":
                    model.add(Conv2D(self.number_of_filters_base, self.filter_size,kernel_initializer = "he_uniform",padding = self.padding))
            
                ## Double filter distribution - double number of filters in each Convolutional layers
                elif self.filter_distribution == "double":
                    model.add(Conv2D(2**(i+1)*self.number_of_filters_base, self.filter_size,kernel_initializer = "he_uniform",padding = self.padding))
            
                ## Halve the filter size in each successive convolutional layers
                elif self.filter_distribution == "half":
                    model.add(Conv2D(int(self.number_of_filters_base/2**(i+1)),self.filter_size, kernel_initializer = "he_uniform"))
            
                if self.batch_normalisation_location == "Before" and self.batch_normalization: model.add(BatchNormalization())
                model.add(Activation(self.activation))
            
                if self.batch_normalisation_location == "After" and self.batch_normalization: model.add(BatchNormalization())
                
                model.add(MaxPooling2D(pool_size=self.pool_size))
            
            #Final densely connected layers
            if self.global_average_pooling == True:
                model.add(GlobalAveragePooling2D())
            else:
                model.add(Flatten())
            model.add(Dense(self.dense_neurons, activation = 'sigmoid'))
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            model.add(Dense(self.num_classes, activation = 'softmax'))
          
            return model 
                 
        
    def load_pretrained_model(self):
        base_model = BASE_MODELS[self.base_model_name]
        base = base_model(weights='imagenet', include_top=False)
        x = base.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(self.dense_neurons, activation='relu')(x)
        guesses = Dense(self.num_classes, activation='softmax')(x)
        model = Model(inputs=base.input, outputs=guesses)

        # freeze all base layers
        for layer in base.layers:
            layer.trainable = False

        #model.compile(optimizer=self.optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model
 

In [52]:
# data preprocessing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import pathlib

#wandb logging
import wandb
from wandb.keras import WandbCallback


#physical_devices = tf.config.list_physical_devices('GPU')
#try:
#  tf.config.experimental.set_memory_growth(physical_devices[0], True)
#except:
 # Invalid device or cannot modify virtual devices once initialized.
#  pass


IMG_SIZE = (128,128)


#sweep config
sweep_config = {
  "name": "Bayesian Sweep",
  "method": "bayes",
  "metric":{
  "name": "val_accuracy",
  "goal": "maximize"
  },
  'early_terminate': {
        'type':'hyperband',
        'min_iter': [3],
        's': [2]
  },
  "parameters": {
        
        "activation":{
            "values": ["relu", "elu", "selu"]
        },
        "filter_size": {
            "values": [(2,2), (3,3), (4,4)]
        },
        "batch_size": {
            "values": [32, 64]
        },
        "padding": {
            "values": ["same","valid"]
        },
        "data_augmentation": {
            "values": [True, False]
        },
        "optimizer": {
            "values": ["sgd", "adam", "rmsprop", "nadam"]
        },
        "batch_normalization": {
            "values": [True, False]
        },
        "batch_normalisation_location": {
            "values": ["Before", "After"]
        },
        "number_of_filters_base": {
            "values": [32, 64]
        },
        "dense_neurons": {
            "values": [32, 64, 128]
        },   
        "dropout_location": {
            "values": ["conv","dense","all"]
        },
        "dropout_fraction": {
            "values": [None, 0.2,0.3]
        },  
        "global_average_pooling": {
            "values": [False,True]
        },        
    }
}

sweep_id = wandb.sweep(sweep_config,project='CS6910-Assignment2-CNNs', entity='rahulsundar')



#train function
def train():

        
    config_defaults = dict(
            num_hidden_cnn_layers = 5 ,
            activation = 'relu',
            batch_normalization = True,
            batch_normalisation_location = "After",
            filter_distribution = "double" ,
            filter_size = (3,3),
            number_of_filters_base  = 32,
            dropout_fraction = None,
            dropout_location = "dense",
            pool_size = (2,2),
            padding = 'same',
            dense_neurons = 128,
            num_classes = 10,
            optimizer = 'adam',
            epochs = 5,
            batch_size = 32, 
            data_augmentation = False,
            global_average_pooling = True,
            img_size = IMG_SIZE
        ) 



    wandb.init( config = config_defaults)
    #wandb.init(project = 'CS6910-Assignment2-CNNs', config = config_defaults,entity='rahulsundar')
    CONFIG = wandb.config


    wandb.run.name = "OBJDET_" + str(CONFIG.num_hidden_cnn_layers) + "_dn_" + str(CONFIG.dense_neurons) + "_opt_" + CONFIG.optimizer + "_dro_" + str(CONFIG.dropout_fraction) + "_bs_"+str(CONFIG.batch_size) + "_fd_" + CONFIG.filter_distribution + "_bnl_" + CONFIG.batch_normalisation_location + "_dpl_" + CONFIG.dropout_location

    data_augmentation = CONFIG.data_augmentation

    
    BATCH_SIZE = CONFIG.batch_size


    if data_augmentation == True:

    #Faster Alternative
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                validation_split = 0.1,
                shear_range=0.2,
                zoom_range=0.2,
                featurewise_center=False,  # set input mean to 0 over the dataset
                samplewise_center=False,  # set each sample mean to 0
                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                samplewise_std_normalization=False,  # divide each input by its std
                zca_whitening=False,  # apply ZCA whitening
                rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
                width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                horizontal_flip=True,  # randomly flip images
                vertical_flip=False
                )
    else:
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split = 0.1)

    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


    train_generator = train_datagen.flow_from_directory(
        './inaturalist_12K/train',
        subset='training',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle = True,
        seed = 123)
        
    validation_generator = train_datagen.flow_from_directory(
            './inaturalist_12K/train',
            target_size=IMG_SIZE,
            subset = 'validation',
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            shuffle = True,
            seed = 123)


            
    test_generator = test_datagen.flow_from_directory(
            './inaturalist_12K/test',
            target_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            shuffle = True,
            seed = 123)



    with tf.device('/device:GPU:0'):        
        objDetn = ObjectDetection(CONFIG.img_size, CONFIG )
        if CONFIG.dropout_location == "all":
            model = objDetn.build_cnnmodel_all()
        elif CONFIG.dropout_location == "conv":
            model = objDetn.build_cnnmodel_conv()
        elif CONFIG.dropout_location == "dense":
            model = objDetn.build_cnnmodel_dense()
        
        model.summary()



        model.compile(
        optimizer=CONFIG.optimizer,  # Optimizer
        # Loss function to minimize
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),#'categorical_crossentropy',
        # List of metrics to monitor
        metrics=['accuracy'],
        )
      
        history = model.fit(
                        train_generator,
                        steps_per_epoch = train_generator.samples // CONFIG.batch_size,
                        validation_data = validation_generator, 
                        validation_steps = validation_generator.samples // CONFIG.batch_size,
                        epochs = CONFIG.epochs, 
                        callbacks=[WandbCallback()]
                        )

        model.save('./TrainedModel/'+wandb.run.name)
        wandb.finish()
        return model, history
        


Create sweep with ID: l1f34o6o
Sweep URL: https://wandb.ai/rahulsundar/CS6910-Assignment2-CNNs/sweeps/l1f34o6o


In [23]:
#train()

In [ ]:
wandb.agent(sweep_id, train, count = 100)

wandb: Agent Starting Run: q59nlkm2 with config:
wandb: 	activation: selu
wandb: 	batch_normalisation_location: Before
wandb: 	batch_normalization: True
wandb: 	batch_size: 64
wandb: 	data_augmentation: False
wandb: 	dense_neurons: 128
wandb: 	dropout_fraction: 0.2
wandb: 	dropout_location: dense
wandb: 	filter_size: [4, 4]
wandb: 	global_average_pooling: True
wandb: 	number_of_filters_base: 32
wandb: 	optimizer: nadam
wandb: 	padding: valid


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 125, 125, 32)      1568      
_________________________________________________________________
batch_normalization (BatchNo (None, 125, 125, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 125, 125, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 59, 59, 64)        32832     
_________________________________________________________________
batch_normalization_1 (Batch (None, 59, 59, 64)        256  

epoch,4
loss,1.63159
accuracy,0.43901
val_loss,1.90936
val_accuracy,0.34375
_runtime,551
_timestamp,1618434800
_step,4
best_val_loss,1.90539
best_epoch,2


epoch,▁▃▅▆█
loss,█▆▄▃▁
accuracy,▁▃▄▆█
val_loss,█▅▁▂▁
val_accuracy,▁▅▇▇█
_runtime,▁▃▅▆█
_timestamp,▁▃▅▆█
_step,▁▃▅▆█


wandb: Agent Starting Run: 0biqo39y with config:
wandb: 	activation: relu
wandb: 	batch_normalisation_location: Before
wandb: 	batch_normalization: True
wandb: 	batch_size: 32
wandb: 	data_augmentation: False
wandb: 	dense_neurons: 64
wandb: 	dropout_fraction: 0.3
wandb: 	dropout_location: dense
wandb: 	filter_size: [3, 3]
wandb: 	global_average_pooling: True
wandb: 	number_of_filters_base: 64
wandb: 	optimizer: nadam
wandb: 	padding: same


...

Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 64)      1792      
_________________________________________________________________
activation (Activation)      (None, 126, 126, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 128)       73856

epoch,4
loss,1.87764
accuracy,0.34747
val_loss,1.92751
val_accuracy,0.32188
_runtime,534
_timestamp,1618470391
_step,4
best_val_loss,1.92751
best_epoch,4


epoch,▁▃▅▆█
loss,█▅▄▂▁
accuracy,▁▄▆▇█
val_loss,█▆▄▃▁
val_accuracy,▁▃▆▇█
_runtime,▁▃▅▆█
_timestamp,▁▃▅▆█
_step,▁▃▅▆█


wandb: Agent Starting Run: 1rg3vysc with config:
wandb: 	activation: selu
wandb: 	batch_normalisation_location: After
wandb: 	batch_normalization: True
wandb: 	batch_size: 64
wandb: 	data_augmentation: False
wandb: 	dense_neurons: 64
wandb: 	dropout_fraction: 0.3
wandb: 	dropout_location: all
wandb: 	filter_size: [3, 3]
wandb: 	global_average_pooling: True
wandb: 	number_of_filters_base: 64
wandb: 	optimizer: rmsprop
wandb: 	padding: valid


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 64)      1792      
_________________________________________________________________
activation (Activation)      (None, 126, 126, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 128)       73856